In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# importing necessary modules

### Geocoding API is a service that allows you to convert addresses into a geographic coordinates which you can use to place markers on a map. This service is currently available for the USA only. 

### This can be achieved in batch mode i.e., multiple addresses at once and convert them to geographic coordinates. The compatible file formats to accomplish this task includes .csv, .txt, .dat, and .xlsx files. 

In [ ]:
def get_coordinates(address):
    base_url = "https://geocoding.geo.census.gov/geocoder/locations/onelineaddress"
    params = {
        "address": address,
        "benchmark": "4", # Public_AR_Current
        "format": "json"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        matches = data.get("result", {}).get("addressMatches", [])
        if matches:
            coordinates = matches[0]["coordinates"]
            return coordinates["y"], coordinates["x"]
    return None, None

### Using .txt file as an example with names of random Museums in the United States as an example. 

In [ ]:
# Read the text file
file_path = ....
df = pd.read_csv(file_path, delimiter=???, header=None)  # assuming "\t" tab-delimited file , replace with "," if comma seperated 

# Rename Columns if Needed
df.columns = ["", ""]

In [ ]:
# Get coordinates for each address
df["Latitude"], df["Longitude"] = zip(*df["????"].apply(get_coordinates)) # get the coordinates based on address

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:4326")

In [ ]:
# display the data
df

In [ ]:
# Create map
m = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=4)

# Add markers for each location
for _, row in df.iterrows():
    if not np.isnan(row["Latitude"]) and not np.isnan(row["Longitude"]):
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=row["Address"],
            tooltip=row["Address"]
        ).add_to(m)

In [ ]:
m

## Using Open Street Map to map out POI

### Function to automate query in Open Streeet Map using Overpass Api. 
More information about how to geocode with OSM can be found here. https://wiki.openstreetmap.org/wiki/Overpass_API?form=MG0AV3#Introduction

#### Function to pass the query to Overpass API

#### For correct Query Input visit: https://wiki.openstreetmap.org/wiki/Map_features

##### Store names like "Target", "Wallmart" wouldn't work in this case. We have to be more generic or specify Brand. 

In [ ]:
def get_locations(categories, queries, cities, state, country, brand=None):
    """Fetch locations from OpenStreetMap using Overpass API."""
    overpass_url = "http://overpass-api.de/api/interpreter"
    all_locations = []

    # Construct the optional brand filter
    brand_filter = f'["brand"="{brand}"]' if brand else ''

    # Handle multiple categories, queries, and cities
    if not categories or not queries or not cities:
        print("No categories, queries, or cities provided.")
        return []

    for category in categories:
        for city in cities:
            for query in queries:
                overpass_query = f"""
                [out:json];
                area[name="{city}"]->.searchArea;
                (
                  node["{category}"="{query}"]{brand_filter}(area.searchArea);
                  way["{category}"="{query}"]{brand_filter}(area.searchArea);
                  relation["{category}"="{query}"]{brand_filter}(area.searchArea);
                );
                out center;
                """
                
                try:
                    response = requests.get(overpass_url, params={'data': overpass_query})
                    response.raise_for_status()
                    data = response.json()
                    all_locations.extend(data.get("elements", []))
                except requests.exceptions.RequestException as e:
                    print(f"Request error for {category}={query} in {city}: {e}")
                except requests.exceptions.JSONDecodeError:
                    print(f"Error decoding JSON response from API for {category}={query} in {city}.")

    return all_locations

In [ ]:
def plot_locations(data, city, state, country):
    """Plot locations on a Folium map."""
    if not data:
        print("No locations found.")
        return None

    # Extract the first valid location for map centering
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            m = folium.Map(location=[lat, lon], zoom_start=12)
            break
    else:
        print("No valid coordinates found.")
        return None

    # Add markers
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            name = place.get('tags', {}).get('name', 'Unknown')
            folium.Marker([lat, lon], popup=f"{name} ({lat}, {lon})").add_to(m)

    return m

In [ ]:
def display_locations(data):
    """Display location names with coordinates in a DataFrame."""
    locations = []
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            name = place.get('tags', {}).get('name', 'Unknown')
            locations.append([name, lat, lon])
    
    df = pd.DataFrame(locations, columns=['Name', 'Latitude', 'Longitude'])
    return df

In [ ]:
# Example Query Parameters
category = ...
queries = [....]
cities = [.....]
state = ""
country = ""
brand = None 

# Fetch Data
data = get_locations(category, queries, cities, state, country, brand)

# Plot Data on Map
map_result = plot_locations(data, cities[0] if cities else None, state, country)

# Display DataFrame of Locations
df_locations = display_locations(data)

# Display the map and data
if map_result:
    display(map_result)

In [ ]:
df_locations